In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as plio
import datetime as dt
import requests
from dataverk import Datapackage, Client
from dataverk_vault import api as vault_api
import os

In [ ]:
vault_api.set_secrets_as_envs()
user = os.environ["GH_REPO_USER"]
token = os.environ["GH_REPO_TOKEN"]

In [ ]:
repo_types = ['internal', 'public', 'private']

In [ ]:
def get_repos(repo_type, user=user, token=token):
    params = {'accept': 'application/vnd.github.v3+json'}
    page = 1
    get_more_repos = True
    repos = []
    while get_more_repos:
        url = f'https://api.github.com/orgs/navikt/repos?per_page=100&type={repo_type}&page={page}'
        print(f'Fetching {repo_type} repos, page {page}')
        r = requests.get(url, auth=(user, token))
        repos = repos + [repo['full_name'] for repo in r.json()]
        page = page + 1
        if(len(r.json())<100):
            get_more_repos = False
        
    if len(repos)>0:
        df = pd.DataFrame(repos)
        df.columns = ['repo']
        df['repo_type'] = repo_type
    else:
        df = pd.DataFrame()
    return df

In [ ]:
df = pd.DataFrame()
for repo_type in repo_types:
    repos = get_repos(repo_type)
    if len(repos)>0:
        df = df.append(repos)

In [ ]:
df_repos = df.groupby('repo_type').size().reset_index(name='antall')

In [ ]:
print(df_repos.to_string(index=False))

In [ ]:
andel_public = round(df_repos[df_repos['repo_type']=='public']['antall'].sum() / df_repos['antall'].sum(),3)

In [ ]:
oppdatert = dt.datetime.now().strftime('%d.%m.%Y kl. %H:%M:%S')
today = dt.datetime.now().strftime('%Y-%m-%d')

In [ ]:
os_kpi = go.Figure()

title = 'Grad av åpen kildekode'
subtitle = 'Andel public repo i github.com/navikt'

os_kpi.add_trace(
    go.Indicator(
            mode = 'number',
            value = andel_public,
            number = {'valueformat': '%', 'font': {'size': 150}},
            title = {'text': f'''{title}<br><span style='font-size:0.6em;color:gray;'>{subtitle}</span>''', 
                     'font': {'size': 30}}
            
        )
)


os_kpi.update_layout(
    annotations = [
        go.layout.Annotation(
            showarrow=False,
            text='<span style=\'font-size: 0.8em\'> Sist oppdatert: {}</span>'.format(oppdatert),
            xanchor='right',
            x=1,
            yanchor='top',
            y=0.01
        )       
    ]
)

os_kpi

In [ ]:
fig_repo = go.Figure()
fig_repo.add_trace(go.Bar(
    x = df_repos['repo_type'],
    y = df_repos['antall'],
    name = 'Antall repo',
    marker_color='#920000'
))

fig_repo.update_layout(
    title=go.layout.Title(text = 'Antall repo per type i github.com/navikt<br><span style=\'font-size: 0.5em\'> Sist oppdatert: {}</span>'.format(oppdatert)),
    separators=', ',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title_x=0.5,    
)


fig_repo.update_xaxes(showline=True, linewidth=1, linecolor='black', fixedrange=True)
fig_repo.update_yaxes(showline=True, linewidth=1, linecolor='black', fixedrange=True, tickformat=',g', gridcolor='#E6E6E6')

fig_repo

In [ ]:
metadata = {
    'title': 'Åpen kildekode på navikt-org på GitHub',
    'description': 'Viser antall repositories i [navikt-organisasjonen på GitHub](https://github.com/navikt/), fordelt på repository-typer. Hensikten er å overvåke andelen repositories som er åpen kildekode.',
    'readme': '',
    'accessRights': 'Open', 
    'issued': dt.datetime.now().isoformat(),
    'modified': dt.datetime.now().isoformat(), 
    'language': 'Norsk', 
    'periodicity': '',
    'temporal': {'from': f'2020-11-02', 'to': f'{today}'},
    'author': 'Gøran Berntsen',
    'publisher': {'name': 'nais', 'url': 'https://nais.io'}, # Optional. Default NAV
    'contactpoint': {'name': 'Gøran Berntsen', 'email': 'goran.berntsen@nav.no'},
    'license': {'name': 'CC BY 4.0', 'url': 'http://creativecommons.org/licenses/by/4.0/deed.no'}, #Optional. Default CC BY 4.0
    'keywords': [],
    'spatial': '',
    'theme': [],
    'type': 'datapackage',
    'format': 'datapackage',
    'category': 'category',
    'provenance': 'NAV',
    'repo': '',
    'notebook': '',
    'store': 'nais',
    'project': 'odata',
    'bucket': 'nav-opendata'
}
dp = Datapackage(metadata)

In [ ]:
dp.add_view(spec_type = 'plotly',
            description = '',
            name = 'Åpen kildekode',
            title = 'Åpen kildekode',
            spec = plio.to_json(os_kpi),
            resources = '')

In [ ]:
dp.add_view(spec_type = 'plotly',
            description = '''Merk at de ulike typene er beskrevet slik på GitHub:
            *Public repositories are accessible to everyone on the internet. Private repositories are only accessible to you, people you explicitly share access with, and, for organization repositories, certain organization members. Internal repositories are accessible to enterprise members.*
            ''',
            name = 'Antall repositories',
            title = 'Antall repositories',
            spec = plio.to_json(fig_repo),
            resources = '')

In [ ]:
dv = Client()

In [ ]:
dv.context.settings["index_connections"]["elastic_host"] = "https://dv-api-ekstern.prod-gcp.nais.io/index/write/dcat"
os.environ["DATAVERK_BUCKET_ENDPOINT"] = "https://dv-api-ekstern.prod-gcp.nais.io/storage"
dv.publish(dp)